In [1]:
import notebook_setup

import jax.numpy as jnp
import pandas as pd

from faker import Faker

In [2]:
from src.entities.weekly_patterns import WeeklyPattern, SchedulesWeeklyPattern
from src.entities.data_classes import Shift
from src.entities.shifts import SchedulesShift

fk = Faker()

weekly_patterns = [
    WeeklyPattern(id=f"p{idx}", pattern=p)
    for idx, p in enumerate(
        [
            [True, True, True, True, True, False, False],
            [True, True, True, True, False, False, True],
            [True, True, True, False, False, True, True],
            [True, True, False, False, True, True, True],
            [True, False, False, True, True, True, True],
            [False, False, True, True, True, True, True],
            [False, True, True, True, True, True, False],
            [True, False, True, True, True, True, False],
            [True, True, False, True, True, True, False],
            [True, True, True, False, True, True, False],
            [True, True, False, True, True, False, True],
            [True, False, True, True, True, False, True],
            [False, True, True, True, True, False, True],
            [False, True, True, True, False, True, True],
            [False, True, True, False, True, True, True],
            [True, False, True, True, False, True, True],
        ]
    )
]

shifts = [
    SchedulesShift(
        Shift(id="m1", start_time=7.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="m2", start_time=8.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="m3", start_time=9.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a1", start_time=14.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a2", start_time=15.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="a3", start_time=17.0, duration=9.0, task="base"), type="work"
    ),
    SchedulesShift(
        Shift(id="n", start_time=22.0, duration=9.0, task="base"), type="work"
    ),
]

Develop `_generate_possible_patterns`

In [3]:
off_shift = SchedulesShift(
    Shift(id="off", start_time=0, duration=24.0, task="off"), type="off"
)

blocks_gap: int = 1

single_day_off_start_time_variance: float = 2.0
multi_day_off_start_time_variance: float = 8.0

shifts = shifts
weekly_patterns = weekly_patterns
possible_patterns = []

for weekly_pattern in weekly_patterns:
    pattern = weekly_pattern.pattern
    blocks, edges = weekly_pattern.blocks(blocks_gap)
    if blocks == 1:
        for shift in shifts:
            shift_list = [shift.id if is_work else off_shift.id for is_work in pattern]
            compound_id = f"{weekly_pattern.id}_{shift.id}"
            possible_patterns.append(
                SchedulesWeeklyPattern(
                    id=compound_id, pattern=pattern, shift_list=shift_list
                )
            )

    else:
        for edge in edges:
            consec_off = list.index(pattern[edge:], True)
            for first_shift in shifts:
                for second_shift in shifts:
                    diff_start = abs(second_shift.start_time - first_shift.start_time)
                    if (
                        consec_off == 1
                        and abs(diff_start) > single_day_off_start_time_variance
                    ):
                        print(f"invalid shift combination (1 off)")
                    elif diff_start > multi_day_off_start_time_variance:
                        print(f"invalid shift combination (multi off)")
                    else:
                        shift_list = [
                            *[
                                first_shift.id if is_work else off_shift.id
                                for is_work in pattern[0:edge]
                            ],
                            *[
                                second_shift.id if is_work else off_shift.id
                                for is_work in pattern[edge:]
                            ],
                        ]

                        compound_id = (
                            f"{weekly_pattern.id}_{first_shift.id}_{second_shift.id}"
                        )

                        possible_patterns.append(
                            SchedulesWeeklyPattern(
                                id=compound_id, pattern=pattern, shift_list=shift_list
                            )
                        )

invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift combination (1 off)
invalid shift 

Develop `_calculate_schedules_residuals`

In [4]:
from conftest import TestObjects
from src.generators.schedules_generator import (
    SchedulesGenerator,
    SchedulesRequirement,
    SchedulesAgent,
    SchedGeneratorParams,
    Schedules,
)

from src.entities.shifts import RequirementShift, SchedulesShift, WeeklyShiftSequence
from src.entities.data_classes import Shift

db_shifts = [
    Shift(id="A", start_time=9.0, duration=9.0, task="task1"),
    Shift(id="B", start_time=12.0, duration=9.0, task="task1"),
    Shift(id="C", start_time=16.0, duration=9.0, task="task1"),
    Shift(id="D", start_time=23.0, duration=9.0, task="task1"),
]

req_shifts = [RequirementShift(s) for s in db_shifts]

sched_shifts = [SchedulesShift.from_reqs_shift(s, "work") for s in req_shifts]

# 2 weeks of requirement
values = jnp.array(
    [
        [3, 2, 2, 2, 2, 1, 1],
        [4, 3, 3, 3, 3, 2, 2],
        [2, 2, 2, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
    ]
)

sched_reqs = SchedulesRequirement(req_shifts, values)

agents = [
    SchedulesAgent("A1", tasks=["task1", "task2"]),
    SchedulesAgent("A2", tasks=["task1", "task2"]),
    SchedulesAgent("A3", tasks=["task1", "task2"]),
    SchedulesAgent("A4", tasks=["task1", "task2"]),
    SchedulesAgent("A5", tasks=["task1", "task2"]),
    SchedulesAgent("A6", tasks=["task1"]),
    SchedulesAgent("A7", tasks=["task1"]),
    SchedulesAgent("A8", tasks=["task1"]),
]

vac = SchedulesShift.new_all_day_shift(id="vac", task="vac", type="pto")
off = SchedulesShift.new_off_shift()
unav_16 = SchedulesShift.new_unavailable_shift(0, 16)
task2_9 = SchedulesShift.new_work_shift("Z", 9.0, 9.0, "task2")
task1_12 = sched_shifts[1]

availability_dict = {
    "A1": [None, None, unav_16, vac, vac, off, off],
    "A2": [task1_12, task1_12, task1_12, None, None, off, off],
    "A3": [off, off, None, None, None, None, None],
    "A5": [None, None, task2_9, off, None, None, None],
    "A8": [vac, vac, off, off, None, None, None],
}

Developing ``generate``

In [5]:
schedules = SchedulesGenerator.setup_schedules(agents, availability_dict)
sched_gen = SchedulesGenerator(requirements=sched_reqs, schedules=schedules)

In [6]:
schedules[0].scheduled_shifts.get_weekly_shifts(0)
task1_12

SchedulesShift(id='B', start_time=12.0, duration=9.0, task='task1')

In [7]:
week = 0
sched_coverage = []
for shift in sched_reqs.shifts:
    sched_coverage.append(
        sum(
            [
                jnp.array(
                    sched.scheduled_shifts.get_weekly_shift_coverage(week, [shift.id])
                ).astype(int)
                for sched in schedules
            ]
        ),
    )

jnp.array(sched_coverage)

Array([[0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [8]:
# Get Current Schedules Coverage
def get_multi_shift_weekly_coverage(
    schedules: Schedules, week, shifts: list[Shift]
) -> jnp.ndarray:
    coverage = []

    for shift in shifts:
        coverage.append(
            schedules.scheduled_shifts.get_weekly_shift_coverage(week, [shift.id])
        )

    return coverage

In [ ]:
# Single Pattern Coverage
possible_patterns = sched_gen._generate_possible_patterns()
jnp.array(
    [possible_patterns[0].get_shift_coverage(shift) for shift in sched_shifts]
).astype(int)

Array([[1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
sched_gen.requirements.get_weekly_values(1)

Array([[3, 2, 2, 2, 2, 1, 1],
       [4, 3, 3, 3, 3, 2, 2],
       [2, 2, 2, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)